## HW 5 - Page Rank
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2023`__

> Updated: 06/30/2023

Last homework for 261!!!

In Weeks 8 and 9 you discussed key concepts related to graph based algorithms and implemented SSSP. In this final homework assignment you'll implement distributed PageRank using some data from Wikipedia. You will likely want to select 16 or 32 cores when you set up your cluster, as graph processing with Spark can take some time. If you are working on free trial credits, then you maximum will be 16 cores. You'll also need to enable GraphFrames when you spin up your cluster or you won't be able to complete the last question.

By the end of this homework you should be able to:  
* ... __compare/contrast__ adjacency matrices and lists as representations of graphs for parallel computation.
* ... __explain__ the goal of the PageRank algorithm using the concept of an infinite Random Walk.
* ... __define__ a Markov chain including the conditions underwhich it will converge.
* ... __identify__ what modifications must be made to the web graph inorder to leverage Markov Chains.
* ... __implement__ distributed PageRank in Spark.

### Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

### Start Spark and the Spark UI

In [2]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import os

try:
    spark
except NameError:
    print('starting Spark')
    app_name = 'HW5_notebook'
    master = "local[*]"
    spark = SparkSession\
            .builder\
            .appName(app_name)\
            .master(master)\
            .getOrCreate()
sc = spark.sparkContext

ZONE = os.getenv("ZONE") #new as of June 17, 2022 4:45PM PST; restart cluster to see thus env variable
spark_UI_info = f'''
To start the SPARK UI

Copy the following command  to cloud shell and run as shown in the screenshot below. 
And click on the PREVIEW ON PORT 8080 menu option .
Run the following command to establish the SSH Tunnel:

gcloud compute ssh w261-m --zone {ZONE} --ssh-flag "-L 8080:localhost:4040" '''

print(spark_UI_info)

starting Spark
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
graphframes#graphframes added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-968f4d1e-13ea-47a5-9f2a-e3dc4a935c8e;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.1-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
	found org.apache.spark#spark-avro_2.12;3.1.3 in central
	found org.spark-project.spark#unused;1.0.0 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.1-s_2.12/graphframes-0.8.2-spark3.1-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.1-s_2.12!graphframes.jar (231ms)
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.1.3/spark-avro_2.12-3.1.3.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-avro_2.12;3.1.3!spark-avro_2.12.jar (16ms)
downloading https://repo1.maven.org/m


To start the SPARK UI

Copy the following command  to cloud shell and run as shown in the screenshot below. 
And click on the PREVIEW ON PORT 8080 menu option .
Run the following command to establish the SSH Tunnel:

gcloud compute ssh w261-m --zone us-west1-b --ssh-flag "-L 8080:localhost:4040" 


<img src='https://www.dropbox.com/s/tlb4uiakj2bx7qg/Three_steps_launch_SparkUI.png?raw=true' style='width:100%'>

## Set up global variables for cloud buckets (both Data, Staging) and for Zone 

In [3]:
import os
from google.cloud import storage
DATA_BUCKET = os.getenv('DATA_BUCKET','')[:-1] # our private storage bucket location
HW5_FOLDER = f"{DATA_BUCKET}/notebooks/jupyter/Assignments/HW5"
print(f"Personal Data bucket:  {DATA_BUCKET}")
!gsutil ls -lh  {HW5_FOLDER} 

Personal Data bucket:  gs://w261-hr
      34 B  2023-07-22T20:51:32Z  gs://w261-hr/notebooks/jupyter/Assignments/HW5/HW5.md
     631 B  2023-07-22T20:51:32Z  gs://w261-hr/notebooks/jupyter/Assignments/HW5/README.md
 56.63 KiB  2023-07-22T20:51:34Z  gs://w261-hr/notebooks/jupyter/Assignments/HW5/hw5_workbook.ipynb
                                 gs://w261-hr/notebooks/jupyter/Assignments/HW5/data/
TOTAL: 3 objects, 58651 bytes (57.28 KiB)


In [ ]:
sc = spark.sparkContext
spark

# Question 1: Distributed Graph Processing
Chapter 5 from Lin & Dyer gave you a high level introduction to graph algorithms and concernts that come up when trying to perform distributed computations over them. The questions below are designed to make sure you captured the key points from this reading and your async lectures. 

### Q1 Tasks:

* __a) Multiple Choice:__ Other than their size/scale, what makes graphs uniquely challenging to work with in the map-reduce paradigm? *(__HINT__: Don't think in terms of any specific algorithm. Think in terms of the nature of the graph data structure itself).*

* __b) Multiple Choice:__ What specific design component in Dijkstra's algorithm makes this approach hard to parallelize?

* __c) Short Response:__ How does parallel breadth-first-search get around the problem that you identified in part `b`? At what expense? Your response should be 2-4 sentences. 

### Q1 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!  

# Question 2: Representing Graphs 

In class you saw examples of adjacency matrix and adjacency list representations of graphs. These data structures were probably familiar from HW3, though we hadn't before talked about them in the context of graphs. In this question we'll discuss some of the tradeoffs associated with these representations. __`NOTE:`__ We'll use the graph from Figure 5.1 in Lin & Dyer as a toy example. For convenience in the code below we'll label the nodes `A`, `B`, `C`, `D`, and `E` instead of $n_1$, $n_2$, etc but otherwise you should be able to follow along & check our answers against those in the text.


<img src="https://github.com/kyleiwaniec/w261_assets/blob/master/images/HW5/Lin-Dyer-graph-Q1.png?raw=true" width=50%>

### Q2 Tasks:

* __a) Multiple Dropdown:__ Relatively speaking, is the graph you described in Figure 5.1 in Lin & Dyer "sparse" or "dense"?  Explain how sparsity/density impacts the adjacency matrix and adjacency list representations of a graph. 

* __b) Multiple Dropdown:__ Run the provided code to create and plot our toy graph. Is this graph directed or undirected? Explain how the adjacency matrices for directed graphs will differ from those of undirected graphs.

* __c) Code:__ Fill in the missing code to complete the function `get_adj_matr()`.

* __d) Code:__ Fill in the missing code to complete the function `get_adj_list()`.

* __e) Multiple Choice:__ Which is the correct edge list for node `B`?

### Q2 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __e)__ Type your answer here!

In [ ]:
# part a - a graph is just a list of nodes and edges (RUN THIS CELL AS IS)
TOY_GRAPH = {'nodes':['A', 'B', 'C', 'D', 'E'],
             'edges':[('A', 'B'), ('A', 'D'), ('B', 'C'), ('B', 'E'), ('C', 'D'), 
                      ('D', 'E'), ('E', 'A'),('E', 'B'), ('E', 'C')]}

In [ ]:
# part a - simple visualization of our toy graph using nx (RUN THIS CELL AS IS)
G = nx.DiGraph()
G.add_nodes_from(TOY_GRAPH['nodes'])
G.add_edges_from(TOY_GRAPH['edges'])
display(nx.draw(G, pos=nx.circular_layout(G), with_labels=True, alpha = 0.5))

In [ ]:
# part c - adjacency matrix function
def get_adj_matr(graph):
    """
    Function to create an adjacency matrix representation of a graph.
    arg:
        graph - (dict) of 'nodes' : [], 'edges' : []
    returns:
        pd.DataFrame with entry i,j representing an edge from node i to node j
    """
    n = len(graph['nodes'])
    adj_matr = pd.DataFrame(0, columns = graph['nodes'], index = graph['nodes'])
    ############### YOUR CODE HERE ##################
    
    ############### (END) YOUR CODE #################
    return adj_matr

In [ ]:
# part c - take a look (RUN THIS CELL AS IS)
TOY_ADJ_MATR = get_adj_matr(TOY_GRAPH)
print(TOY_ADJ_MATR)

In [ ]:
# part d - adjacency list function
def get_adj_list(graph):
    """
    Function to create an adjacency list representation of a graph.
    arg:
        graph - (dict) of 'nodes' : [], 'edges' : []
    returns:
        dictionary of the form {node : [list of edges]}
    """
    adj_list = {node: [] for node in graph['nodes']}
    ############### YOUR CODE HERE ##################

    
    ############### (END) YOUR CODE #################
    return adj_list

In [ ]:
# part d - take a look (RUN THIS CELL AS IS)
TOY_ADJ_LIST = get_adj_list(TOY_GRAPH)
print(TOY_ADJ_LIST)

# Question 3: Markov Chains and Random Walks

As you know from your readings and in class discussions, the PageRank algorithm takes advantage of the machinery of Markov Chains to compute the relative importance of a webpage using the hyperlink structure of the web (we'll refer to this as the 'web-graph'). A Markov Chain is a discrete-time stochastic process. The stochastic matrix has a principal left eigen vector corresponding to its largest eigen value which is one. A Markov chain's probability distribution over its states may be viewed as a probability vector. This steady state probability for a state is the PageRank of the corresponding webpage. In this question we'll briefly discuss a few concepts that are key to understanding the math behind PageRank. 

General note: Q3 and Q4 use a sped-up convergence method for the `power_iteration()` function. You should speed up convergence by multiplying by the tMatrix twice. `tMatrix = tMatrix.dot(tMatrix)`. So instead of:
```
Step 0: xInit * tMatrix
Step 1: xInit * tMatrix^2
Step 2: xInit * tMatrix^3
Step 3: xInit * tMatrix^4
...
```
you should instead consider
```
Step 0: xInit * tMatrix
Step 1: xInit * tMatrix^2
Step 2: xInit * tMatrix^4
Step 3: xInit * tMatrix^8
...
```
This may vary slightly from what you see in Demo 10


### Q3 Tasks:

* __a) Multiple Choice:__ It is common to explain PageRank using the analogy of a web surfer who visits a page, randomly clicks a link on that page, and repeats ad infinitum. In the context of this hypothetical infinite random walk across web pages on the internet, which of the following choices most clearly describes the event that the __teleportation__ represents?

* __b) Multiple Dropdowns:__ What is the "Markov Property" and what does it mean in the context of PageRank?

* __c) Multiple Dropdowns:__ A Markov chain consists of $n$ states plus an $n\times n$ transition probability matrix. In the context of PageRank & a random walk over the WebGraph, what are the $n$ states? what implications does this have about the size of the transition matrix?

* __d) Multiple Choice:__ What is a "right stochastic matrix"?

* __e) Code + Numerical Answer:__ Fill in the code below to compute the transition matrix for the toy graph from question 2. What is the value in the middle column of the last row (the probability of transitioning from node `E` to `C`)? Include the leading digit in front of the decimal, and round the number to at least 4 decimal places. Examples: 1.0000 or 0.1234. [__`HINT:`__ _It should be right stochastic. Using numpy this calculation can be done in one line of code._]

* __f) Code:__ To compute the stable state distribution (i.e. PageRank) of a "nice" graph we can apply the power iteration method - repeatedly multiplying the transition matrix by itself, until the values no longer change. Apply this strategy to your transition matrix from `part d` to find the PageRank for each of the pages in your toy graph. Your code should print the results of each iteration. 
    * __`NOTE 1:`__ _this is a naive approach, we'll unpack what it means to be "nice" in the next question_.
    * __`NOTE 2:`__ _no need to implement a stopping criteria, visual inspection should suffice_.
    * __`NOTE 3:`__ _refer to the General note section above, and use the `tMatrix = tMatrix.dot(tMatrix)` approach for faster convergence_.


* __g) Numerical Answer:__ How many iterations does it take to converge?

* __h) Multiple Choice:__ Which node is the least 'central' (i.e., it has the lowest ranked)?


### Q3 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!  

> __d)__ Type your answer here!  

> __e)__ Type your answer here!  

> __g)__ Type your answer here!  

> __h)__ Type your answer here!  

In [ ]:
# part e - recall what the adjacency matrix looked like (RUN THIS CELL AS IS)
TOY_ADJ_MATR

In [ ]:
# part e - use TOY_ADJ_MATR to create a right stochastic transition matrix for this graph
################ YOUR CODE HERE #################
transition_matrix = None # replace with your code

################ (END) YOUR CODE #################
print(transition_matrix)

In [ ]:
# part f - compute the steady state using the transition matrix 
def power_iteration(xInit, tMatrix, nIter, verbose = True):
    """
    Function to perform the specified number of power iteration steps to 
    compute the steady state probability distribution for the given
    transition matrix.
    
    Args:
        xInit     - (n x 1 array) representing inial state
        tMatrix  - (n x n array) transition probabilities
        nIter     - (int) number of iterations
    Returns:
        state_vector - (n x 1 array) representing probability 
                        distribution over states after nSteps.
    
    NOTE: if the 'verbose' flag is on, your function should print the step
    number and the current matrix at each iteration.
    """
    state_vector = None
    ################ YOUR CODE HERE #################

    
    
    
    ################ (END) YOUR CODE #################
    return state_vector

In [ ]:
# part f - run 10 steps of the power_iteration (RUN THIS CELL AS IS)
xInit = np.array([1.0, 0, 0, 0, 0]) # note that this initial state will not affect the convergence states
states = power_iteration(xInit, transition_matrix, 10, verbose = True)

__`Expected Output for part e after 10 iterations:`__  
>Steady State Probabilities:
```
[0.10526316 0.15789474 0.18421053 0.23684211 0.31578947] 
```

# Question 4: Page Rank Theory

Seems easy right? Unfortunately applying this power iteration method directly to the web-graph actually runs into a few problems. In this question we'll tease apart what we meant by a 'nice graph' in Question 3 and highlight key modifications we'll have to make to the web-graph when performing PageRank. To start, we'll look at what goes wrong when we try to repeat our strategy from question 3 on a 'not nice' graph.

__`Additional References:`__ http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/lecture3.html

### Q4 Tasks:

* __a) code + Multiple Choice:__ Run the provided code to create and plot our 'not nice' graph. Fill in the missing code to compute its transition matrix & run the power iteration method from question 3. What are the first values in steps 1, 2, and 3 of the power iteration method? [__`HINT:`__ _We start the iteration at step number 0. If you start your iteration at step number 1, then you should answer with the values from step 2, 3, and 4 instead_]

* __b) Multiple Choice:__ What is wrong with what you see in part a? [__`HINT:`__ _there is a visible underlying reason that it isn't converging... try adding up the probabilities in the state vector after each iteration._]

* __c) Multiple Choice:__  Identify the dangling node in this 'not nice' graph and explain how this node causes the problem in 'a'. How could we modify the transition matrix after each iteration to prevent this problem?

* __d) Multiple Choice:__ What does it mean for a graph to be irreducible? Is the webgraph naturally irreducible? Choose your reasoning.

* __e) Multiple Choice:__ What does it mean for a graph to be aperiodic? Is the webgraph naturally aperiodic? Choose your reasoning.

* __f) Multiple Choice:__ What modifications to the webgraph does PageRank make in order to guarantee aperiodicity and irreducibility? [__`HINT:`__ _select 2 answers_]

### Q4 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!

> __d)__ Type your answer here!  

> __e)__ Type your answer here!

> __f)__ Type your answer here!

In [ ]:
# part a - run this code to create a second toy graph (RUN THIS CELL AS IS)
TOY2_GRAPH = {'nodes':['A', 'B', 'C', 'D', 'E'],
              'edges':[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'D'), 
                       ('B', 'E'), ('C', 'A'), ('C', 'E'), ('D', 'B')]}

In [ ]:
# part a - simple visualization of our test graph using nx (RUN THIS CELL AS IS)
G = nx.DiGraph()
G.add_nodes_from(TOY2_GRAPH['nodes'])
G.add_edges_from(TOY2_GRAPH['edges'])
display(nx.draw(G, pos=nx.circular_layout(G), with_labels=True, alpha = 0.5))

In [ ]:
# part a - run 10 steps of the power iteration method here
# HINT: feel free to use the functions get_adj_matr() and power_iteration() you wrote above
################ YOUR CODE HERE #################
adj_matr = None # replace with your code
trans_matr = None # replace with your code

state = None # replace with your code
################ (END) YOUR CODE #################

# Question 5: EDA part 1 (number of nodes)

## About the Data
The main dataset for this data consists of a subset of a 500GB dataset released by AWS in 2009. The data includes the source and metadata for all of the Wikimedia wikis. You can read more here: 
> https://aws.amazon.com/blogs/aws/new-public-data-set-wikipedia-xml-data. 

Use the cells below to download the wikipedia data and a test file for use in developing your PageRank implementation(note that we'll use the 'indexed out' version of the graph) and to take a look at the files.

In [ ]:
HW5_FOLDER

In [ ]:
!gsutil ls -l {HW5_FOLDER}

In [ ]:
!gsutil ls -l {HW5_FOLDER}/data

In [ ]:
# load the data into Spark RDDs for convenience of use later (RUN THIS CELL AS IS)
testRDD = sc.textFile(HW5_FOLDER + '/data/test_graph.txt')
indexRDD = sc.textFile(HW5_FOLDER + '/data/indices/')
wikiRDD = sc.textFile(HW5_FOLDER + '/data/all-pages-indexed-out/')

In [ ]:
# display testRDD (RUN THIS CELL AS IS)
testRDD.take(10)

In [ ]:
# display indexRDD (RUN THIS CELL AS IS)
indexRDD.take(10)

In [ ]:
# display wikiRDD (RUN THIS CELL AS IS)
wikiRDD.take(10)

### Q5 Tasks

As usual, before we dive in to the main analysis, we'll peform some exploratory data anlysis to understand our dataset. Please use the test graph that you downloaded to test all your code before running the full dataset.

* __a) Matching:__ What is the format of the raw data? What does the first value represent? What does the second part of each line represent? [__`HINT:`__ _no need to go digging here, just visually inspect the outputs of the head commands that we ran after loading the data above._]

* __b) Multiple Choice:__ Run the provided bash command to count the number of records in the raw dataset. Explain why this is _not_ the same as the number of total nodes in the graph.

* __c) code:__ In the space provided below write a Spark job to count the _total number_ of nodes in this graph. 

* __d) Numeric:__ How many dangling nodes are there in this wikipedia graph? [__`HINT:`__ _you should not need any code to answer this question._]

### Q5 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here! 

> __b)__ Type your answer here! 

> __d)__ Type your answer here! 

In [ ]:
# part b - count the number of records in the raw data (RUN THIS CELL AS IS)
# 5781290
print(wikiRDD.count())

In [ ]:
# part c - write your Spark job here (compute total number of nodes)
def count_nodes(dataRDD):
    """
    Spark job to count the total number of nodes.
    Returns: integer count 
    """    
    ############## YOUR CODE HERE ###############
    totalCount = None  # replace with your code
    
    
    
    ############## (END) YOUR CODE ###############   
    return totalCount

In [ ]:
# part c - run your counting job on the test file (RUN THIS CELL AS IS)
start = time.time()
tot = count_nodes(testRDD)
print(f'... completed job in {time.time() - start} seconds.')
print(f'Total Nodes: {tot}')

In [ ]:
# part c - run your counting job on the full file (RUN THIS CELL AS IS)
start = time.time()
tot = count_nodes(wikiRDD)
print(f'... completed job in {time.time() - start} seconds.')
print(f'Total Nodes: {tot}')

# Question 6 - EDA part 2 (out-degree distribution)

As you've seen in previous homeworks the computational complexity of an implementation depends not only on the number of records in the original dataset but also on the number of records we create and shuffle in our intermediate representation of the data. The number of intermediate records required to update PageRank is related to the number of edges in the graph. In this question you'll compute the average number of hyperlinks on each page in this data and visualize a distribution for these counts (the out-degree of the nodes). 

### Q6 Tasks:

* __a1) code:__ In the space provided below write a Spark job to stream over the data and compute all of the following information:
  * Count the number of out-degree for each __non-dangling node__ and return the names of the top 10 pages with the most hyperlinks
  * Find the average out-degree for all __non-dangling nodes__ in the graph
  * Take a 1000 point sample of these out-degree counts and plot a histogram of the result. 
<br>
<br>
 
* __a2) Numeric:__ What is the average out-degree of the `testRDD`?

* __a3) Multiple Choice:__ What is the top node by out-degree of the `wikiRDD`?

* __b) Multiple Dropdown:__ In the context of the PageRank algorithm, how is information about a node's out degree used?

* __c) Multiple Answer:__ What does it mean if a node's out-degree is 0? In PageRank how will we handle these nodes differently than others? Select all that apply. [__Hint:__ _select 3 answers_]
 
__`NOTE:`__ Please observe scalability best practices in the design of your code & comment your work clearly. You will be graded on both the clarity and the design.

### Q6 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a2)__ Type your answer here!

> __a3)__ Type your answer here!

> __b)__ Type your answer here! 

> __c)__ Type your answer here!

In [ ]:
# part a - write your Spark job here (compute average in-degree, etc)
def count_degree(dataRDD, n):
    """
    Function to analyze out-degree of nodes in a a graph.
    Returns: 
        top  - (list of 10 tuples) nodes with most edges
        avgDegree - (float) average out-degree for non-dangling nodes
        sampledCounts - (list of integers) out-degree for n randomly sampled non-dangling nodes
    """
    # helper func
    def parse(line):
        node, edges = line.split('\t')
        return (node, ast.literal_eval(edges))
    
    ############## YOUR CODE HERE ###############
    
    
    top = None  # replace with your code
    avgDegree = None  # replace with your code
    sampledCounts = None  # replace with your code
    ############## (END) YOUR CODE ###############
    
    return top, avgDegree, sampledCounts

In [ ]:
# part a - run your job on the test file (RUN THIS CELL AS IS)
start = time.time()
test_results = count_degree(testRDD,10)
print(f"... completed job in {time.time() - start} seconds")
print("Average out-degree: ", test_results[1])
print("Top 10 nodes (by out-degree:)\n", test_results[0])

In [ ]:
# part a - plot results from test file (RUN THIS CELL AS IS)
plt.hist(test_results[2], bins=10)
plt.title("Distribution of Out-Degree")
display(plt.show())

In [ ]:
# part a - run your job on the full file (RUN THIS CELL AS IS)
start = time.time()
full_results = count_degree(wikiRDD,1000)

print(f"... completed job in {time.time() - start} seconds")
print("Average out-degree: ", full_results[1])
print("Top 10 nodes (by out-degree:)\n", full_results[0])

In [ ]:
# part a - plot results from full file (RUN THIS CELL AS IS)
plt.hist(full_results[2], bins=50)
plt.title("Distribution of Out-Degree")
display(plt.show())

# Question 7 - PageRank part 1 (Initialize the Graph)

One of the challenges of performing distributed graph computation is that you must pass the entire graph structure through each iteration of your algorithm. As usual, we seek to design our computation so that as much work as possible can be done using the contents of a single record. In the case of PageRank, we'll need each record to include a node, its list of neighbors and its (current) rank. In this question you'll initialize the graph by creating a record for each dangling node and by setting the initial rank to 1/N for all nodes. 

__`NOTE:`__ Your solution should _not_ hard code $N$.

### Q7 Tasks:
* __a) Multiple Choice:__ What is $N$? 

* __b) Multiple Selection:__ Using the analogy of the infinite random web-surfer, how do we use $\frac{1}{N}$? [__HINT:__ _select 2 answers_]

* __c) True or False:__ Will it be more efficient to compute \\(N\\) before initializing records for each dangling node or after? Select True or False for each of the statements.

* __d) Code:__ Fill in the missing code below to create a Spark job that:
  * parses each input record
  * creates a new record for any dangling nodes and sets it edges (neighbors) to be a empty [__HINT:__ _you should use the same data type with the edges for non-dangling nodes_]
  * initializes a rank of 1/N for each node
  * returns a pair RDD with records in the format specified by the docstring


* __e) Numerical Answer:__ Run the provided code to confirm that your job in `part d` has a record for each node. Your records should match the format specified in the docstring and the count should match what you computed in question 5. Then answer the question: how many edges does the node `13415942` have? [__`TIP:`__ _you might want to take a moment to write out what expected outputs you should get for the test graph, this will help you know your code works as expected_]
 
__`NOTE:`__ Please observe scalability best practices in the design of your code & comment your work clearly. You will be graded on both the clarity and the design.

### Q7 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here! 

> __e)__ Type your answer here!

In [ ]:
# part d - job to initialize the graph
def initGraph(dataRDD):
    """
    Spark job to read in the raw data and initialize an 
    adjacency list representation with a record for each
    node (including dangling nodes).
    
    Returns: 
        graphRDD -  a pair RDD of (node_id , (score, edges))
        
    NOTE: The score should be a float, but you may want to be 
    strategic about how format the edges... there are a few 
    options that can work. Make sure that whatever you choose
    is sufficient for Question 8 where you'll run PageRank.
    """
    ############## YOUR CODE HERE ###############

    # write any helper functions here
    
    
    
    
    
    
    
    
    
    # write your main Spark code here
    graphRDD = None # replace with your code
    
    
    
    
    ############## (END) YOUR CODE ##############
    
    return graphRDD

In [ ]:
# part e - run your Spark job on the test graph (RUN THIS CELL AS IS)
start = time.time()
testGraph = initGraph(testRDD).collect()
print(f'... test graph initialized in {time.time() - start} seconds.')
testGraph

In [ ]:
# part e - run your code on the main graph (RUN THIS CELL AS IS)
# Takes about 100 seconds
start = time.time()
wikiGraphRDD = initGraph(wikiRDD)
print(f'... full graph initialized in {time.time() - start} seconds')

In [ ]:
# part e - confirm record format and count (RUN THIS CELL AS IS)
start = time.time()
print(f'Total number of records: {wikiGraphRDD.count()}')
node = wikiGraphRDD.sortByKey().lookup(13415942) 
print("Node 13415942's initialized record:", node)
print(f"\n\nNode 13415942 has {len(node[0][1].split(','))} outlinks.")
print(f'Time to complete: {time.time() - start} seconds.')

# Question 8 - PageRank part 2 (Iterate until convergence)

Finally we're ready to compute the page rank. In this last question you'll write a Spark job that iterates over the initialized graph updating each nodes score until it reaches a convergence threshold. The diagram below gives a visual overview of the process using a 5 node toy graph. Pay particular attention to what happens to the dangling mass at each iteration.

<img src='https://github.com/kyleiwaniec/w261_assets/blob/master/images/HW5/PR-illustrated.png?raw=true' width=50%>



__`A Note about Notation:`__ The formula above describes how to compute the updated page rank for a node in the graph. The $P$ on the left hand side of the equation is the new score, and the $P$ on the right hand side of the equation represents the accumulated mass that was re-distributed from all of that node's in-links. Finally, $|G|$ is the number of nodes in the graph (which we've elsewhere refered to as $N$).

### Q8 Tasks:
* __a) Multiple Answers:__ In terms of the infinite random walk analogy, interpret the meaning of the first term in the PageRank calculation: $\alpha * \frac{1}{|G|}$. 
    [__Hint__: _select two answers_]

* __b) Multiple Choice:__ In the equation for the PageRank calculation above what does $m$ represent and why do we divide it by $|G|$?

* __c) Numerical Answer:__ Keeping track of the total probability mass after each update is a good way to confirm that your algorithm is on track. How much should the total mass be after each iteration?

* __d) Code & Multiple Choice:__ Fill in the missing code below to create a Spark job that take the initialized graph as its input then iterates over the graph and for each pass:
  * reads in each record and redistributes the node's current score to each of its neighbors
  * uses an accumulator to add up the dangling node mass and redistribute it among all the nodes. (_Don't forget to reset this accumulator after each iteration!_)
  * uses an accumulator to keep track of the total mass being redistributed.( _This is just for your own check, its not part of the PageRank calculation. Don't forget to reset this accumulator after each iteration._)
  * aggregates these partial scores for each node
  * applies telportation and damping factors as described in the formula above.
  * combine all of the above to compute the PageRank as described by the formula above.
  <br>

  Then answer the question: What is the top ranked node of the `wikiaGraphRDD`?
    
> TIP: You can check your work for part `d` by looking at the nodes in the top 20-40th positions by PageRank score, which should match the expected output.

   __WARNING:__ Some pages contain multiple hyperlinks to the same destination, please take this into account when redistributing the mass.

 
__`NOTE:`__ Please observe scalability best practices in the design of your code & comment your work clearly. You will be graded on both the clarity and the design.

### Q8 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here!

> __b)__ Type your answer here! 

> __c)__ Type your answer here!

The following cell can be used as additonal info / background knowledge on how to create a custom accumulator class. **YOU DO NOT NEED TO USE THIS CLASS IN ORDER TO COMPLETE THE ASSIGNMENT.**

In [ ]:
#  *** OPTIONAL.. SEE NOTE ABOVE ***
# part d - provided FloatAccumulator class (RUN THIS CELL AS IS) 

from pyspark.accumulators import AccumulatorParam

class FloatAccumulatorParam(AccumulatorParam):
    """
    Custom accumulator for use in page rank to keep track of various masses.
    
    IMPORTANT: accumulators should only be called inside actions to avoid duplication.
    We stringly recommend you use the 'foreach' action in your implementation below.
    """
    def zero(self, value):
        return value
    def addInPlace(self, val1, val2):
        return val1 + val2

In [ ]:
# part d - job to run PageRank
def runPageRank(graphInitRDD, alpha = 0.15, maxIter = 10, verbose = True):
    """
    Spark job to implement page rank
    Args: 
        graphInitRDD  - pair RDD of (node_id , (score, edges))
        alpha         - (float) teleportation factor
        maxIter       - (int) stopping criteria (number of iterations)
        verbose       - (bool) option to print logging info after each iteration
    Returns:
        steadyStateRDD - pair RDD of (node_id, pageRank)
    """
    # teleportation:
    a = sc.broadcast(alpha)
    
    # damping factor:
    d = sc.broadcast(1-a.value)
    
    # initialize accumulators for dangling mass & total mass
    mmAccum = sc.accumulator(0.0, FloatAccumulatorParam())
    totAccum = sc.accumulator(0.0, FloatAccumulatorParam())
    
    ############## YOUR CODE HERE ###############
    
    # write your helper functions here, 
    # please document the purpose of each clearly 
    # for reference, the master solution has 5 helper functions.


            
    
    
    
    
    
    
    
    
    
    
    
    
    
        
    # write your main Spark Job here (including the for loop to iterate)
    # for reference, the master solution is 21 lines including comments & whitespace
    steadyStateRDD = None  # replace with your code
    
    
    
    
    
    
    
    
    
    
    ############## (END) YOUR CODE ###############
    
    return steadyStateRDD

In [ ]:
# part d - run PageRank on the test graph (RUN THIS CELL AS IS)
# NOTE: while developing your code you may want turn on the verbose option
nIter = 20
testGraphRDD = initGraph(testRDD)
start = time.time()
test_results = runPageRank(testGraphRDD, alpha = 0.15, maxIter = nIter, verbose = False)
print(f'...trained {nIter} iterations in {time.time() - start} seconds.')
print(f'Top 20 ranked nodes:')
test_results.takeOrdered(20, key=lambda x: - x[1])

---
__`expected results for the test graph:`__
```
[(2, 0.3620640495978871),
 (3, 0.333992700474142),
 (5, 0.08506399429624555),
 (4, 0.06030963508473455),
 (1, 0.04255740809817991),
 (6, 0.03138662354831139),
 (8, 0.01692511778009981),
 (10, 0.01692511778009981),
 (7, 0.01692511778009981),
 (9, 0.01692511778009981),
 (11, 0.01692511778009981)]
```
---

In [ ]:
# part d - run PageRank on the full graph (RUN THIS CELL AS IS)
# NOTES: 
# - wikiGraphRDD should have been computed & cached above!
# - Rounding errors after 10 or so decimal places are acceptable
# - This will take a long time to run...about 30 minutes on 16 cores
nIter = 10
start = time.time()
full_results = runPageRank(wikiGraphRDD, alpha = 0.15, maxIter = nIter, verbose = True)
print(f'\n...trained {nIter} iterations in {time.time() - start} seconds.\n')

print(f'Top 20-40th ranked nodes:\n')
top_40 = full_results.takeOrdered(40, key=lambda x: - x[1])
# print results from 20th to 40th highest PageRank
for result in top_40[20:]:
    print(result)

---
**Expected output for pages in top 20-40:**
```
(9386580, 0.0002956970419566159)
(15164193, 0.0002843018514565407)
(7576704, 0.00028070286946292773)
(12074312, 0.0002789390949918953)
(3191491, 0.0002733179890615315)
(2797855, 0.0002728954946638793)
(11253108, 0.000272424060446348)
(13725487, 0.00027020945537652716)
(2155467, 0.0002664929816794768)
(7835160, 0.00026125677808592145)
(4198751, 0.00025892998307531953)
(9276255, 0.0002546338645363715)
(10469541, 0.0002537014715361682)
(11147327, 0.00025154645090952246)
(5154210, 0.00024927029620654557)
(2396749, 0.0002481158912959703)
(3603527, 0.0002465136725657213)
(3069099, 0.00023478481004084842)
(12430985, 0.0002334928892642041)
(9355455, 0.00022658004696908508)
```
 ---

In [ ]:
# view record from indexRDD (RUN THIS CELL AS IS)
# title\t indx\t inDeg\t outDeg
indexRDD.take(1)

In [ ]:
# map indexRDD to new format (index, name) (RUN THIS CELL AS IS)
namesKV_RDD = indexRDD.map(lambda x: (int(x.split('\t')[1]), x.split('\t')[0]))

In [ ]:
# see new format (RUN THIS CELL AS IS)
namesKV_RDD.take(2)

In [ ]:
# part d - What is the top ranked node of the wikiaGraphRDD - (RUN THIS CELL AS IS)
top_40_RDD = sc.parallelize(top_40)
top_40_RDD.take(1)

# Question 9 - Pretty Print

### Q9 Tasks
The indexRDD we created earlier from the indices.txt file contains the titles of the pages and their IDs.

* __a) Code:__ Join this dataset with your top 40 results. The print statement provided will display the pages in the top 20-40th positions.

* __b) Fill in the blank:__ The title of the page associated with the PageRank score of 0.000279 is _____.

### Q9 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __b)__ Type your answer here! 

Join with indexRDD and print pretty

In [ ]:
# part a
############## YOUR CODE HERE ###############
joinedWithNames = None   # replace with your code
############## END YOUR CODE ###############

In [ ]:
# part b
# Feel free to modify this cell to suit your implementation, but please keep the formatting and sort order.
print("{:10s}\t| {:10s}\t| {}".format("PageRank","Page id","Title"))
print("="*100)
for r in joinedWithNames[20:]:
    print ("{:6f}\t| {:10d}\t| {}".format(r[1][1],r[0],r[1][0]))

# Question 10 - GraphFrames
GraphFrames is a graph library which is built on top of the Spark DataFrames API. If you're doing graph-based analysis in the future, this is a great tool that you can use. GraphFrames does calculations like PageRank is about 25% of the time that you can achieve using the map-reduce paradigm you've used up to this point. The goal of this final question is to expose you to this tool so that you at least have some knowledge of its basic functionality.

### Q10 Tasks
* __a) code + Multiple Choice:__ Load in the Les Misérables data. Which character has their only edge connected to MmeBurgon?
* __b) code + Multiple Choice:__ Complete the code to run the GraphFrames implementation of PageRank and report out the top 10 characters. Which character had the third highest PageRank value?

### Q10 Student Answers:
OPTIONAL - if you'd like to save your answers in this notebook: 

> __a)__ Type your answer here! 

> __b)__ Type your answer here! 

In [ ]:
!pip install graphframes

In [ ]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from graphframes import *
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

In [ ]:
# part a - (RUN CELL AS IS)

# load in the les miserable network 
G = nx.les_miserables_graph()

# take a look at the graph (you can RUN CELL AS IS or feel free adjust the view)
fig, ax = plt.subplots(figsize=(16, 16)) 
nx.draw(G, with_labels=True, ax=ax, node_color='#4bcffa')

In [ ]:
# creates the vertices and edges (RUN CELL AS IS)
vertices = spark.createDataFrame(list(G.nodes), StringType())
edges = spark.createDataFrame(list(G.edges))

vertices = vertices.select(vertices.value.alias('id')).distinct()

edges = edges.withColumnRenamed('_1','src')
edges = edges.withColumnRenamed('_2','dst')

In [ ]:
# take a look at the vertices (RUN CELL AS IS)
vertices.show(10)

In [ ]:
# take a look at the edges (RUN CELL AS IS)
edges.show(10)

In [ ]:
# part b
# Create a GraphFrame
f = GraphFrame(vertices, edges)

# TODO: call the pageRank algorithm, with resetProbability=0.15 and maxIter=10
# Don't worry about the warnings
############## YOUR CODE HERE ###############
results = None
############## END YOUR CODE ###############

In [ ]:
# part b (continued)

# TODO: order the results in descending order and limit to the top 10 characters
start = time.time()
############## YOUR CODE HERE ###############
top_10 = None  # replace with your code. HINT: top_10 = results.vertices...

############## END YOUR CODE ###############
print(f'... completed job in {time.time() - start} seconds.')

In [ ]:
%%time
# part b (continued)
top_10.show()

### Congratulations, you have completed the last homework for 261! 